In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import os
import pymongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
url = 'https://mars.nasa.gov/news/'
html = requests.get(url)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url2 = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ethom\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [4]:
db = client.mission_to_mars_db
collection = db.scrape

In [5]:
soup = BeautifulSoup(html.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- Responsiveness -->
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <!-- Favicon -->
  <link href="/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/manifest.json" rel="manifest"/>
  <link color="#e48b55" href="/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="#000000" name="theme-color"/>
  <meta content="authenticity_token" name="csrf-param">
   <meta content="2FEB0lrGI+XubqgLzzj4dD0+SX+Swp/g985yT5YiHxXN4kV3AbL5ALedyPgzWM0N8CJgPbaqTp+m0edgbYlVZQ==" name="csrf-to

In [6]:
results = soup.find_all('div', class_='features')

In [7]:
for result in results:
    
    title = result.find('div', class_='content_title')
    
    try:
       
        news_title = title.text
        news_p = result.a.div.text

        if (news_title or news_p):
            print('\n-----------------\n')
            print('Title:', news_title)
            print('Description:', news_p)
            article = {
                'Title': news_title,
                'Description': news_p,
            }
            collection.insert_one(article)
    except AttributeError as e:
        print(e)


-----------------

Title: 

NASA Ingenuity Mars Helicopter Prepares for First Flight


Description: 

Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  







In [26]:
html2 = browser.html
soup = BeautifulSoup(html2, 'html.parser')

feats = soup.find('img', class_='headerimage fade-in')
link = feats['src']
featured_image_url = {'featured_image_url': f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{link}'}
collection.insert_one(featured_image_url)
return featured_image_url

{'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg', '_id': ObjectId('605cfaa08d72529282555e66')}


In [21]:
url3 = 'https://space-facts.com/mars/'

In [28]:
import json
tables = pd.read_html(url3)

print(tables)

[                      0                              1
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers,   Mars - Earth Comparison             Mars            Earth
0               Diameter:         6,779 km        12,742 km
1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
2                  Moons:                2                1
3      Distance from Sun:   227,943,824 km   149,598,262 km
4         Length of Year:   687 Earth days      365.24 days
5            Temperature:     -87 to -5 °C      -88 to 58°C,                       0    

In [33]:
df = tables[0]
df = df.to_json()

json.dumps(df)
# collection.insert_one(df)


'"{\\"0\\":{\\"0\\":\\"Equatorial Diameter:\\",\\"1\\":\\"Polar Diameter:\\",\\"2\\":\\"Mass:\\",\\"3\\":\\"Moons:\\",\\"4\\":\\"Orbit Distance:\\",\\"5\\":\\"Orbit Period:\\",\\"6\\":\\"Surface Temperature:\\",\\"7\\":\\"First Record:\\",\\"8\\":\\"Recorded By:\\"},\\"1\\":{\\"0\\":\\"6,792 km\\",\\"1\\":\\"6,752 km\\",\\"2\\":\\"6.39 \\\\u00d7 10^23 kg (0.11 Earths)\\",\\"3\\":\\"2 (Phobos & Deimos)\\",\\"4\\":\\"227,943,824 km (1.38 AU)\\",\\"5\\":\\"687 days (1.9 years)\\",\\"6\\":\\"-87 to -5 \\\\u00b0C\\",\\"7\\":\\"2nd millennium BC\\",\\"8\\":\\"Egyptian astronomers\\"}}"'

In [34]:
url4 = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
html3 = requests.get(url4)

In [35]:
soup = BeautifulSoup(html3.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | USGS Astrogeology Science Center
  </title>
  <meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verification"/>
  <!--<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,bold"/>-->
  <link href="/css/main.css" media="screen" rel="stylesheet"/>
  <link href="/css/print.css" media="pr

In [36]:
results2 = soup.find_all('div', class_='item')

In [40]:
hemisphere_image_url = []
for result in results2:
    
    title = result.find('h3').text
    link = result.a['href']
    img_url = result.img['src']
    
    hemisphere_image_url={"Title": title, "img_url": link,img_url}
    collection.insert_one(hemisphere_image_url)
    return hemisphere_image_url

{'Title': 'Cerberus Hemisphere Enhanced', 'img_url': '/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png', '_id': ObjectId('605d00128d72529282555e67')}
{'Title': 'Schiaparelli Hemisphere Enhanced', 'img_url': '/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png', '_id': ObjectId('605d00128d72529282555e68')}
{'Title': 'Syrtis Major Hemisphere Enhanced', 'img_url': '/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png', '_id': ObjectId('605d00128d72529282555e69')}
{'Title': 'Valles Marineris Hemisphere Enhanced', 'img_url': '/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png', '_id': ObjectId('605d00128d72529282555e6a')}


In [ ]:
! jupyter nbconvert --to script mission_to_mars.ipynb

In [ ]:
browser.quit()